# Building an Agent with Long-Term Memory using `FlotorchMemoryService`

This notebook demonstrates how to give an agent a long-term memory. We will use the `FlotorchMemoryService` to store and retrieve information across different, independent sessions. This allows the agent to build a persistent knowledge base over time.

### Prerequesit
Configure model, memory provider and API key in Flotroch console (https://console.flotorch.cloud/)

### Viewing logs
Logs can be viewed in logs tab in Flotroch console (https://console.flotorch.cloud/)

### Key Objectives:
- Configure and integrate the `FlotorchMemoryService`.
- Provide the agent with the `load_memory` tool to access its knowledge base.
- Demonstrate the agent's ability to recall information from a previously completed session.

## 1. Setup and Imports

The following cells handle package installation, API configuration, and importing all necessary modules. The key components for this example are `FlotorchMemoryService` and the `load_memory` tool.

In [ ]:
# install flotorch adk package
%pip install --pre flotorch[adk]

In [ ]:
FLOTORCH_API_KEY = "<flotorch api key>"
FLOTORCH_BASE_URL = "<flotroch gateway base url>" # eg: https://gateway.flotorch.cloud"
FLOTORCH_MODEL = "<flotorch model>"
APP_NAME = "flotorch_tools_example"
USER_ID = "flotorch_user_001"
MEMORY_PROVIDER = "<flotorch memory provider>"

In [ ]:
# import necessary libraries
from flotorch.adk.llm import FlotorchADKLLM
from flotorch.adk.memory import FlotorchMemoryService
from flotorch.adk.sessions import FlotorchADKSession
from google.adk.agents import LlmAgent
from google.adk import Runner
from google.genai import types
from google.adk.tools import load_memory

print("Imported necessary libraries successfully")

## 2. Configuring the Memory and LLM

First, we initialize the `FlotorchMemoryService`, which provides the backend for our agent's long-term storage. We then configure the `FlotorchADKLLM` as usual.

In [ ]:
model = FlotorchADKLLM(
    model_id=FLOTORCH_MODEL,
    api_key=FLOTORCH_API_KEY,
    base_url=FLOTORCH_BASE_URL,
)

memory_service = FlotorchMemoryService(
    name=MEMORY_PROVIDER,
    api_key=FLOTORCH_API_KEY,
    base_url=FLOTORCH_BASE_URL,
)

print(f"LLM and Memory Service are configured.")

## 3. Configuring the Agent and Runner with Memory

This section wires everything together:
1. **Agent**: We create an `LlmAgent` and give it the `load_memory` tool. This tool allows the agent to query its long-term knowledge base.
2. **Runner**: We initialize the `Runner`, passing it both the `FlotorchADKSession` service (for short-term memory) and the `FlotorchMemoryService` (for long-term memory).

In [ ]:
agent = LlmAgent(
    name="Flotorch_Memory_Assistant",
    description="A Flotorch AI assistant with Flotorch Memory Service integration",
    instruction="You are a Flotorch AI assistant with memory capabilities. Use your memory to provide contextual responses.",
    model=model,
    tools=[load_memory],  # Provide the agent with the memory access tool
)

session_service = FlotorchADKSession(
    api_key=FLOTORCH_API_KEY,
    base_url=FLOTORCH_BASE_URL
)

runner = Runner(
    agent=agent,
    app_name=APP_NAME,
    session_service=session_service,
    memory_service=memory_service  # Integrate the long-term memory service
)

print("Runner configured successfully with memory service integration.")

## 4. Chatting with a Memory-Enabled Agent

The following demonstration involves two separate sessions. In the first session, we will tell the agent a fact. After the chat, we explicitly save the session to the long-term memory service. In the second session, we will ask the agent to recall that fact, proving its long-term memory is working.

In [ ]:
def run_single_turn(query, session_id, user_id, runner):
    content = types.Content(role="user", parts=[types.Part(text=query)])
    events = runner.run(user_id=user_id, session_id=session_id, new_message=content)
    for event in events:
        if event.is_final_response():
            if event.content and event.content.parts:
                return event.content.parts[0].text
    return "Sorry, I couldn't process that request."

async def chat_with_agent_and_save_memory(query, session_id):
    response = run_single_turn(query, session_id, USER_ID, runner)
    # After the turn, get the completed session and save it to long-term memory
    completed_session = await runner.session_service.get_session(
        app_name=APP_NAME, user_id=USER_ID, session_id=session_id
    )
    await memory_service.add_session_to_memory(completed_session)
    return response

print("Memory-aware chat functions are ready.")

In [ ]:
# === Session 1: Teach the agent a fact ===
session1 = await runner.session_service.create_session(
    app_name=APP_NAME, user_id=USER_ID,
)
print(f"Started Session 1 with ID: {session1.id}")

response1 = await chat_with_agent_and_save_memory("My name is Vikram and I work as a software engineer.", session1.id)
print(f"\nQuery: My name is Vikram and I work as a software engineer.")
print(f"Response: {response1}")
print("\n--- Session 1 ended and memory saved ---")

In [ ]:
# === Session 2: Start a new conversation and recall the fact ===
session2 = await runner.session_service.create_session(
    app_name=APP_NAME, user_id=USER_ID,
)
print(f"\nStarted Session 2 with ID: {session2.id}")

response2 = await chat_with_agent_and_save_memory("What is my name?", session2.id)
print(f"\nQuery: What is my name?")
print(f"Response: {response2}")

## Summary

This notebook demonstrated how to equip a Google ADK agent with a persistent, long-term memory using `FlotorchMemoryService`. This allows the agent to build and retain knowledge across completely separate user sessions.

### Key Achievements:

- **Memory Integration**: We successfully configured the `Runner` with `FlotorchMemoryService` to handle long-term storage.
- **Knowledge Persistence**: The agent was able to store information from one session and accurately recall it in a new, later session.
- **Advanced Capabilities**: This dual-memory architecture (short-term session context + long-term knowledge base) is crucial for creating truly intelligent and personalized AI assistants that learn over time.